# Reinforcement Learning

Reinforcement learning (RL) is a learning algorithm to find a set of optimal policies for any given state a system can be in. RL is particularly good at problems that do not have an immediate reward, that is, problems in which only time will tell, whether a sequence of actions has been the right one or not. This is very nicely illustrated by the game <a href="https://pygame-learning-environment.readthedocs.io/en/latest/user/games/catcher.html">Catcher</a>, a game even simpler as "Pong". The goal of the game is to catch falling fruit by moving a paddle to the left and right. If the player misses the fruit, the game is lost. 

<center>
<img src="figs/catcher.gif" width="150\%">
</center>

In Catcher, the <i>state</i> is whatever you see on the screen. What the state is, is up to the designer. For example, the state could be a tuple consisting of the (X,Y) position of the falling fruit and the position of the paddle. The state could also be the history of the last four of these numbers. Finally, the state could even be the entire screen itself (64x64 pixels) or the last few screens, e.g. 4x64x64 pixels. This is were deep learning comes in.

The <i>policy</i> given a state is whether to move the paddle to the left, to the right, or not to move at all. The state can therefore be encoded with 3 integers, for example 0, 1, and 2.

We can now imagine to train a classifier that takes the state of the game and produces the appropriate action as an output. In a supervised learning setting, which we are used to so far, this would require an expert to play the game many times, record all state and action pairs and use the final outcome for a sequence of actions as label. How to do this if expert samples are not available, however? This is where RL comes in.  

## Markov Decision Process

The RL problem is commonly modeled as a <a href="https://en.wikipedia.org/wiki/Markov_decision_process">Markov Decision Process</a> (MDP). A MDP is a set of states, each with a set of actions that can lead to different states with a certain probability, and with rewards associated with some states. A MDP reduces to a <a href="https://en.wikipedia.org/wiki/Markov_chain">Markov Chain</a> (MC) if every state has only one action to chose from and if there are no rewards. The key point in both is the so-called "Markov property", which states that a system's future state is fully defined by its current state, not the history thereof. In other words, once a situation has been reduced to a MC or MDP, all planning can be done just given the current state. 

Lets refer to the state of the system at time $t$ as $s_t$, the action the agent is taking as $a_t$, and the reward it is receiving as $r_t$. An episode of a game is therefore given by a sequence of state, action and reward, tuples, or

$$s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \ldots, s_{n-1}, a_{n-1}, r_{n-1}, s_n, a_n, r_n $$

In Catcher, the states will encode the fruit and paddle positions, the actions will be drawn from the set {0,1,2}, and the rewards will be zero except for the last action, which will result in -1 (missed) or +1 (caught). 

## Maximizing future rewards

The total reward of an episode is therefore given by $r_{total}=\sum_i^{\forall n} r_i$. Whenever a new decision to take an action needs to be made, the agent should chose the action that maximizes the <i>future reward</i>.

At every point in time $t$, the agent therefore needs to look into the future (assuming for now some best course of action) and add up all rewards it could get

$$ R_t = \sum_{i=t}^n r_i+r_{i+1}+\ldots+r_n $$

Even if we had a table with the best sequence of actions to move forward from every state, the outcome of each action in a MDP is still random. RL addresses this problem by introducing a discount factor $\gamma$. If $\gamma=1$, we assume the agent can predict the future perfectly (such as in a deterministic scenario). If $\gamma=0$, we assume the agent cannot predict the future at all, but will greedily select the action with the highest immediate reward. The equation for $R_t$ therefore changes to

$$ R_t = \sum_{i=t}^n r_i+\gamma r_{i+1}+\gamma^2 r_{i+2} \ldots+\gamma^{n-t}r_n $$

or, recursively,

$$ R_t = r_t + \gamma R_{t+1}. $$

## Q-Learning

One of the simplest algorithms to solve a RL problem, is known as Q-Learning. Q-learning centers around the so-called "Q-function" that provides the maximum discounted future reward $R_{t}$ for an action-state pair $s_t$ and $a_t$

$$ Q(s_t,a_t)=\max(R_{t}) $$

The learning algorithm now aims at maximizing the Q-function, that is finding the highest possible value for at least one action-state pair. If we indeed had such a function, we would need to simply look up which action $a_t$ would give us the highest reward and chose accordingly. This can be written as

$$ \Pi(s_t) = \arg \max_{a_t} Q(s_t,a_t), $$

that is the "policy" $\Pi(s_t)$ returns the action $a_t$ that maximizes the future reward.

To better understand, how the Q-function works, we can expand it 

$$ Q(s_t,a_t)=r_t + \gamma \max_{a_{t+1}}Q(s_{t+1},a_{t+1}) $$

If we had a way to learn the Q-function, we could create a table with as many rows as there are states and as many columns as there are actions and store the maximum expected discounted reward there. We could then use such a table to pick the best action and then see where the randomness in our MDP drives us next.

One way to learn the Q-function is the <a href="https://en.wikipedia.org/wiki/Bellman_equation">Bellman equation</a>, which provides an update rule for the Q-function that is essentially a weighted average between the previous value and a new observation. Here, the Q-table can be initialized with random or fixed numbers. The update proceeds as follows:

$$Q'(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha (r_t + \gamma \max_{a} Q(s_{t+1},a) -Q(s_t,a_t)) $$

This equation might look overwhelming at first, but can be dissected into logical components. First, the new value of the Q-function is the old one minus $\alpha Q(s_t,a_t)$, which is given by the first and last terms. $\alpha$ is known as the learning rate. If $\alpha$ is zero, the Q-value never changes. If $\alpha=1$, old information is discarded immediately. The Q-value is then updated by the actual reward $r_t$ and the maximum discounted future reward based on the current information in the Q-table. 

Learning the entire Q-table now works as follows:

1. Initizalize the Q-table to random or constant value that is below our usual rewards
2. Observe the initial state s
3. Repeat<br>
 3.1 Select and carry out action $a$<br>
 3.2 Observe reward $r$ and move to new state $s'$<br>
 3.3 $Q(s,a)=Q(s,a)+\alpha(r+\gamma \max_{a'} Q(s',a')-Q(s,a)$<br>
 3.4 $s=s'$
4. until game over

Think about what happens when applied to the game catcher: At first, the reward is always zero. Only at the very end, that is when $s'$ is equivalent to "game over", do we get an reward and can update $Q(s,a)$. Let's say this happens at time $t$. We therefore update $Q(s_{t-1},a_{t-1})$. We have to now run the algorithm one more time, but now we can see the discounted reward that we can get at $s_{t-1}$ already at state $s_{t-2}$. The reward that can only be had at the end of the game therefore backpropagates, eventually becoming available at the beginning of the game, allowing us to eventually make better decisions. The key is therefore step (3.1) that should balance between exploring new states and exploiting what we know from the Q-table. 

## Selecting the next action: balancing exploration and exploitation

This can be accomplished by randomly choosing between taking the best known action and exploring another (random) one. For example, the agent might choose the best known action with probability 1-$\epsilon$, and a random one with probability $\epsilon$. This strategy is known as $\epsilon-$greedy. As the quality of the Q-table will increase over time, it makes sense to adapt the value of $\epsilon$ as training progresses. Specifically, it is good practice to start with $\epsilon=1$, that is all actions are random, and gradually decrease it to 0.1 or lower. If $\epsilon=0$, the algorithm will always chose the best known action. 

## Storing the Q-table in a deep network

RL would be a pretty simple task, would the state be an obvious set of discrete numbers. It is easy to construct a suitable state for the "Catcher" game. We can even think about smart choices, such as the distance between the paddle and the fruit on the x-axis - learning a policy that moves into a direction that minimizes this distance would actually be very simple - but what if we don't want to? In particular for games with more complex graphics or multiple moving parts it might be much more convenient to let a deep neural network do the feature engineering. 

In the case of "Catcher", the input to a deep Q-function could be the last four screens, passed through a dense convolutional network, and performing regression to calculate the discounted reward (the value of the Q-function) for each possible action. Such a network then would have 4x64x64 inputs and 3x1 outputs. The loss function in this case is the mean square error between the output of the network and the reward we are actually getting

$$ L=\frac{1}{2}[r+\gamma \max_{a'} Q(s',a')-Q(s,a)]^2 $$

Here, $Q(s,a)$ is the current prediction of the network, $Q(s',a')$ are the predictions for all possible actions in $s'$, and $r$ is the current reward. 

As this network is getting trained, like in the original algorithm, the reward from the final state will eventually backpropagate through the network. Note that training works on a step-by-step base. The network is trained at every time step with every new action that becomes available (left, right, or do nothing in the case of Catcher). 

## Experience Replay

Learning the Q-table this way is rather slow and has a high chance to get stuck in a local minima by repeating similar action sequences. Instead, we can also take advantage of the fact that we can store all previous states, the actions we took, the reward we got, and the next state we chose. Whenever adding a new observation, we can throw in a random subset of this experience memory. This generates more diversity during training, improving generalization. 